In [21]:
# Celda 1: importar librerías
import requests
import pandas as pd
import time
import json
from sqlalchemy import create_engine, text
from time import sleep
import ast

In [48]:
import psycopg2

try:
    connection = psycopg2.connect(
        database="pokemon_db",
        user="postgres",
        password="1234",
        host="localhost",
        port="5432"
    )
    print("Conexión exitosa")

    cursor = connection.cursor()

    # Resto del código para consultas
except psycopg2.Error as e:
    print(f"Error de conexión: {e}")

Conexión exitosa


In [49]:
# Consulta SELECT
cursor.execute("SELECT * FROM pokemon")
resultados = cursor.fetchall()

columnas = [desc[0] for desc in cursor.description]

In [50]:
# Convertir a DataFrame
df = pd.DataFrame(resultados, columns=columnas)

# Mostrar resultado
df.head()

,id,name,height,weight,base_experience,types,abilities,hp,attack,defense
0,1,bulbasaur,7,69,64,"[""grass"", ""poison""]","[""overgrow"", ""chlorophyll""]",45,49,49
1,2,ivysaur,10,130,142,"[""grass"", ""poison""]","[""overgrow"", ""chlorophyll""]",60,62,63
2,3,venusaur,20,1000,236,"[""grass"", ""poison""]","[""overgrow"", ""chlorophyll""]",80,82,83
3,4,charmander,6,85,62,"[""fire""]","[""blaze"", ""solar-power""]",39,52,43
4,5,charmeleon,11,190,142,"[""fire""]","[""blaze"", ""solar-power""]",58,64,58


In [51]:
df_fact = df[['id', 'name', 'height', 'weight', 'base_experience']]
df_fact.head

<bound method NDFrame.head of         id          name  height  weight  base_experience
0        1     bulbasaur       7      69               64
1        2       ivysaur      10     130              142
2        3      venusaur      20    1000              236
3        4    charmander       6      85               62
4        5    charmeleon      11     190              142
...    ...           ...     ...     ...              ...
1020  1021   raging-bolt      52    4800              295
1021  1022  iron-boulder      15    1625              295
1022  1023    iron-crown      16    1560              295
1023  1024     terapagos       2      65               90
1024  1025     pecharunt       3       3              300

[1025 rows x 5 columns]>

In [52]:
df['height'] = df['height'].astype(int)  
df['weight'] = df['weight'].astype(int)  
df['base_experience'] = df['base_experience'].astype(int)
df['types']= df['types'].apply(ast.literal_eval)
df['abilities'] = df['abilities'].apply(ast.literal_eval)
df['hp'] = df['hp'].astype(int)  
df['attack'] = df['attack'].astype(int)  
df['defense'] = df['defense'].astype(int)  


In [57]:
#convirtiendo type a catalogo
todos_los_tipos = [tipo for sublista in df['types'] for tipo in sublista]
tipos_unicos = sorted(set(todos_los_tipos))
catalogo_tipos = [{'id_type': i + 1, 'tipo_name': tipo} for i, tipo in enumerate(tipos_unicos)]
df_tipos = pd.DataFrame(catalogo_tipos)
df_tipos.head()

,id_type,tipo_name
0,1,bug
1,2,dark
2,3,dragon
3,4,electric
4,5,fairy


In [54]:
#convirtiendo abilities a catalogo
todos_las_habilidades = [tipo for sublista in df['abilities'] for tipo in sublista]
habilidades_unicos = sorted(set(todos_las_habilidades))
catalogo_habilidades = [{'id_abilities': i + 1, 'tipo_name': tipo} for i, tipo in enumerate(habilidades_unicos)]
df__habilidades = pd.DataFrame(catalogo_habilidades)
df__habilidades.head()

,id_abilities,tipo_name
0,1,adaptability
1,2,aftermath
2,3,air-lock
3,4,analytic
4,5,anger-point


In [ ]:
#creando tabla de habilidades
df_habilidades = df[['id', 'abilities']].explode('abilities')
df_habilidades = df_habilidades.rename(columns={'abilities': 'abilitie'})
df_habilidades.head()

,id,habilidad
0,1,overgrow
0,1,chlorophyll
1,2,overgrow
1,2,chlorophyll
2,3,overgrow


In [ ]:
#creando tabla de tipos
df_tipos = df[['id', 'types']].explode('types')
df_tipos = df_tipos.rename(columns={'types': 'type'})
df_tipos.head()

,id,tipos
0,1,grass
0,1,poison
1,2,grass
1,2,poison
2,3,grass
